In [ ]:
import libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sklearn
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import scale
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import RFE
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.preprocessing import scale
Extract libraries
data=pd.read_csv('../input/car-price-prediction/CarPrice_Assignment.csv')
data.head()
car_ID	symboling	CarName	fueltype	aspiration	doornumber	carbody	drivewheel	enginelocation	wheelbase	...	enginesize	fuelsystem	boreratio	stroke	compressionratio	horsepower	peakrpm	citympg	highwaympg	price
0	1	3	alfa-romero giulia	gas	std	two	convertible	rwd	front	88.6	...	130	mpfi	3.47	2.68	9.0	111	5000	21	27	13495.0
1	2	3	alfa-romero stelvio	gas	std	two	convertible	rwd	front	88.6	...	130	mpfi	3.47	2.68	9.0	111	5000	21	27	16500.0
2	3	1	alfa-romero Quadrifoglio	gas	std	two	hatchback	rwd	front	94.5	...	152	mpfi	2.68	3.47	9.0	154	5000	19	26	16500.0
3	4	2	audi 100 ls	gas	std	four	sedan	fwd	front	99.8	...	109	mpfi	3.19	3.40	10.0	102	5500	24	30	13950.0
4	5	2	audi 100ls	gas	std	four	sedan	4wd	front	99.4	...	136	mpfi	3.19	3.40	8.0	115	5500	18	22	17450.0
5 rows × 26 columns

def check(df):
    l=[]
    columns=df.columns
    for col in columns:
        dtypes=df[col].dtypes
        nunique=df[col].nunique()
        sum_null=df[col].isnull().sum()
        l.append([col,dtypes,nunique,sum_null])
    df_check=pd.DataFrame(l)
    df_check.columns=['column','dtypes','nunique','sum_null']
    return df_check 
check(data)
column	dtypes	nunique	sum_null
0	car_ID	int64	205	0
1	symboling	int64	6	0
2	CarName	object	147	0
3	fueltype	object	2	0
4	aspiration	object	2	0
5	doornumber	object	2	0
6	carbody	object	5	0
7	drivewheel	object	3	0
8	enginelocation	object	2	0
9	wheelbase	float64	53	0
10	carlength	float64	75	0
11	carwidth	float64	44	0
12	carheight	float64	49	0
13	curbweight	int64	171	0
14	enginetype	object	7	0
15	cylindernumber	object	7	0
16	enginesize	int64	44	0
17	fuelsystem	object	8	0
18	boreratio	float64	38	0
19	stroke	float64	37	0
20	compressionratio	float64	32	0
21	horsepower	int64	59	0
22	peakrpm	int64	23	0
23	citympg	int64	29	0
24	highwaympg	int64	30	0
25	price	float64	189	0
def check2(df):
    print(len(df.index))
    print(df.shape)
    print('________')
    print(df.describe())
check2(data)
205
(205, 26)
________
           car_ID   symboling   wheelbase   carlength    carwidth   carheight  \
count  205.000000  205.000000  205.000000  205.000000  205.000000  205.000000   
mean   103.000000    0.834146   98.756585  174.049268   65.907805   53.724878   
std     59.322565    1.245307    6.021776   12.337289    2.145204    2.443522   
min      1.000000   -2.000000   86.600000  141.100000   60.300000   47.800000   
25%     52.000000    0.000000   94.500000  166.300000   64.100000   52.000000   
50%    103.000000    1.000000   97.000000  173.200000   65.500000   54.100000   
75%    154.000000    2.000000  102.400000  183.100000   66.900000   55.500000   
max    205.000000    3.000000  120.900000  208.100000   72.300000   59.800000   

        curbweight  enginesize   boreratio      stroke  compressionratio  \
count   205.000000  205.000000  205.000000  205.000000        205.000000   
mean   2555.565854  126.907317    3.329756    3.255415         10.142537   
std     520.680204   41.642693    0.270844    0.313597          3.972040   
min    1488.000000   61.000000    2.540000    2.070000          7.000000   
25%    2145.000000   97.000000    3.150000    3.110000          8.600000   
50%    2414.000000  120.000000    3.310000    3.290000          9.000000   
75%    2935.000000  141.000000    3.580000    3.410000          9.400000   
max    4066.000000  326.000000    3.940000    4.170000         23.000000   

       horsepower      peakrpm     citympg  highwaympg         price  
count  205.000000   205.000000  205.000000  205.000000    205.000000  
mean   104.117073  5125.121951   25.219512   30.751220  13276.710571  
std     39.544167   476.985643    6.542142    6.886443   7988.852332  
min     48.000000  4150.000000   13.000000   16.000000   5118.000000  
25%     70.000000  4800.000000   19.000000   25.000000   7788.000000  
50%     95.000000  5200.000000   24.000000   30.000000  10295.000000  
75%    116.000000  5500.000000   30.000000   34.000000  16503.000000  
max    288.000000  6600.000000   49.000000   54.000000  45400.000000  
Data cleaning
as we see car_ID is not useful,let's drop this column

data.drop('car_ID',axis=1,inplace=True)
let's see the main names of cars

data['CarName'] = data['CarName'].str.split(' ',expand=True)[0]
data.head()
symboling	CarName	fueltype	aspiration	doornumber	carbody	drivewheel	enginelocation	wheelbase	carlength	...	enginesize	fuelsystem	boreratio	stroke	compressionratio	horsepower	peakrpm	citympg	highwaympg	price
0	3	alfa-romero	gas	std	two	convertible	rwd	front	88.6	168.8	...	130	mpfi	3.47	2.68	9.0	111	5000	21	27	13495.0
1	3	alfa-romero	gas	std	two	convertible	rwd	front	88.6	168.8	...	130	mpfi	3.47	2.68	9.0	111	5000	21	27	16500.0
2	1	alfa-romero	gas	std	two	hatchback	rwd	front	94.5	171.2	...	152	mpfi	2.68	3.47	9.0	154	5000	19	26	16500.0
3	2	audi	gas	std	four	sedan	fwd	front	99.8	176.6	...	109	mpfi	3.19	3.40	10.0	102	5500	24	30	13950.0
4	2	audi	gas	std	four	sedan	4wd	front	99.4	176.6	...	136	mpfi	3.19	3.40	8.0	115	5500	18	22	17450.0
5 rows × 25 columns

unique names of cars

data['CarName'].unique()
array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'maxda', 'mazda', 'buick', 'mercury',
       'mitsubishi', 'Nissan', 'nissan', 'peugeot', 'plymouth', 'porsche',
       'porcshce', 'renault', 'saab', 'subaru', 'toyota', 'toyouta',
       'vokswagen', 'volkswagen', 'vw', 'volvo'], dtype=object)
there are some false car names,let's change them to true

data['CarName'] = data['CarName'].replace({'maxda': 'mazda', 'nissan': 'Nissan', 'porcshce': 'porsche', 'toyouta': 'toyota', 
                            'vokswagen': 'volkswagen', 'vw': 'volkswagen'})
data['symboling']=data['symboling'].astype('str')
data['CarName'].unique()
array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'mazda', 'buick', 'mercury', 'mitsubishi',
       'Nissan', 'peugeot', 'plymouth', 'porsche', 'renault', 'saab',
       'subaru', 'toyota', 'volkswagen', 'volvo'], dtype=object)
Let's create categorical columns

categorical_cols=data.select_dtypes(include=['object']).columns
data[categorical_cols].head(2)
symboling	CarName	fueltype	aspiration	doornumber	carbody	drivewheel	enginelocation	enginetype	cylindernumber	fuelsystem
0	3	alfa-romero	gas	std	two	convertible	rwd	front	dohc	four	mpfi
1	3	alfa-romero	gas	std	two	convertible	rwd	front	dohc	four	mpfi
plt.figure(figsize=(12,9))
sns.countplot(x='CarName',data=data)
plt.xticks(rotation=90)
plt.title('Values of Car names')
plt.show()

Encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['CarName'] = le.fit_transform(data['CarName'])
data['fueltype'] = le.fit_transform(data['fueltype'])
data['aspiration'] = le.fit_transform(data['aspiration'])
data['doornumber'] = le.fit_transform(data['doornumber'])
data['carbody'] = le.fit_transform(data['carbody'])
data['drivewheel'] = le.fit_transform(data['drivewheel'])
data['enginelocation'] = le.fit_transform(data['enginelocation'])
data['enginetype'] = le.fit_transform(data['enginetype'])
data['cylindernumber'] = le.fit_transform(data['cylindernumber'])
data['fuelsystem'] = le.fit_transform(data['fuelsystem'])
Data visualization

plt.figure(figsize=(12,8))
sns.set_palette('bright')
plt.title('Car Price Plot')
sns.histplot(data['price'])
plt.show()

plt.figure(figsize=(12,9))
sns.boxplot(x = 'enginetype', y = 'price', data = data,palette='Accent')

plt.show()

plt.figure(figsize=(12,9))
sns.scatterplot(x="wheelbase", y="price", data=data,color='red')
plt.title('price values of wheelbase')
plt.show()

fuel=data['fueltype'].value_counts()
fuel
1    185
0     20
Name: fueltype, dtype: int64
labels=fuel.index
sizes=fuel.values
plt.figure(figsize=(12,9))
colors = sns.color_palette('Paired')
plt.pie(sizes,labels=labels,autopct='%1.1f%%',
        shadow=True,colors=colors,startangle=90)
plt.show()

plt.figure(figsize=(12,9))
sns.distplot(data['price'])
plt.title('price density')
plt.show()

sns.set_palette('dark')
plt.figure(figsize=(16,9))
sns.countplot(x='CarName', hue='fueltype', data=data)
plt.xticks(rotation=90)
plt.show()

data.corr()
CarName	fueltype	aspiration	doornumber	carbody	drivewheel	enginelocation	wheelbase	carlength	carwidth	...	enginesize	fuelsystem	boreratio	stroke	compressionratio	horsepower	peakrpm	citympg	highwaympg	price
CarName	1.000000	-0.065597	0.060091	-0.143542	0.053781	-0.007548	0.060707	0.061688	0.057615	-0.033914	...	-0.157652	0.146779	0.201519	-0.201044	0.086201	-0.091439	-0.143058	0.053777	0.055715	-0.173728
fueltype	-0.065597	1.000000	-0.401397	0.191491	-0.147853	-0.132257	0.040070	-0.308346	-0.212679	-0.233880	...	-0.069594	0.041529	-0.054451	-0.241829	-0.984356	0.163926	0.476883	-0.255963	-0.191392	-0.105679
aspiration	0.060091	-0.401397	1.000000	-0.031792	0.063028	0.066465	-0.057191	0.257611	0.234539	0.300567	...	0.108217	0.288086	0.212614	0.222982	0.295541	0.241685	-0.183383	-0.202362	-0.254416	0.177926
doornumber	-0.143542	0.191491	-0.031792	1.000000	-0.680358	0.098954	0.137757	-0.447357	-0.398568	-0.207168	...	-0.020742	0.015519	-0.119258	0.011082	-0.177888	0.126947	0.247668	0.012417	0.036330	-0.031835
carbody	0.053781	-0.147853	0.063028	-0.680358	1.000000	-0.155745	-0.277009	0.401362	0.334433	0.131710	...	-0.073352	-0.065079	0.010549	-0.015325	0.136243	-0.153928	-0.109643	0.031697	-0.007170	-0.083976
drivewheel	-0.007548	-0.132257	0.066465	0.098954	-0.155745	1.000000	0.147865	0.459745	0.485649	0.470751	...	0.524307	0.424686	0.481827	0.071591	0.127479	0.518686	-0.039417	-0.449581	-0.452220	0.577992
enginelocation	0.060707	0.040070	-0.057191	0.137757	-0.277009	0.147865	1.000000	-0.187790	-0.050989	-0.051698	...	0.196826	0.105971	0.185042	-0.138455	-0.019762	0.317839	0.198461	-0.153487	-0.102026	0.324973
wheelbase	0.061688	-0.308346	0.257611	-0.447357	0.401362	0.459745	-0.187790	1.000000	0.874587	0.795144	...	0.569329	0.384601	0.488750	0.160959	0.249786	0.353294	-0.360469	-0.470414	-0.544082	0.577816
carlength	0.057615	-0.212679	0.234539	-0.398568	0.334433	0.485649	-0.050989	0.874587	1.000000	0.841118	...	0.683360	0.557810	0.606454	0.129533	0.158414	0.552623	-0.287242	-0.670909	-0.704662	0.682920
carwidth	-0.033914	-0.233880	0.300567	-0.207168	0.131710	0.470751	-0.051698	0.795144	0.841118	1.000000	...	0.735433	0.521434	0.559150	0.182942	0.181129	0.640732	-0.220012	-0.642704	-0.677218	0.759325
carheight	0.158837	-0.284631	0.087311	-0.552208	0.568534	-0.019719	-0.106234	0.589435	0.491029	0.279210	...	0.067149	0.017046	0.171071	-0.055307	0.261214	-0.108802	-0.320411	-0.048640	-0.107358	0.119336
curbweight	-0.026969	-0.217275	0.324902	-0.197379	0.128467	0.575111	0.050468	0.776386	0.877728	0.867032	...	0.850594	0.611642	0.648480	0.168790	0.151362	0.750739	-0.266243	-0.757414	-0.797465	0.835305
enginetype	-0.163265	0.082695	-0.102963	0.062431	-0.037024	-0.116823	0.114127	-0.135577	-0.113291	0.012298	...	0.040766	-0.091787	0.029355	-0.141918	-0.071873	0.010301	0.005599	-0.085004	-0.078456	0.049171
cylindernumber	0.008118	0.110617	-0.133119	0.154322	-0.048408	0.223238	0.135541	-0.184596	-0.109585	-0.194074	...	-0.085613	0.011970	-0.032844	-0.050088	-0.064701	0.115612	0.222731	-0.126422	-0.085897	-0.027628
enginesize	-0.157652	-0.069594	0.108217	-0.020742	-0.073352	0.524307	0.196826	0.569329	0.683360	0.735433	...	1.000000	0.514070	0.583774	0.203129	0.028971	0.809769	-0.244660	-0.653658	-0.677470	0.874145
fuelsystem	0.146779	0.041529	0.288086	0.015519	-0.065079	0.424686	0.105971	0.384601	0.557810	0.521434	...	0.514070	1.000000	0.475599	0.088153	-0.100786	0.655638	0.014261	-0.671581	-0.645659	0.526823
boreratio	0.201519	-0.054451	0.212614	-0.119258	0.010549	0.481827	0.185042	0.488750	0.606454	0.559150	...	0.583774	0.475599	1.000000	-0.055909	0.005197	0.573677	-0.254976	-0.584532	-0.587012	0.553173
stroke	-0.201044	-0.241829	0.222982	0.011082	-0.015325	0.071591	-0.138455	0.160959	0.129533	0.182942	...	0.203129	0.088153	-0.055909	1.000000	0.186110	0.080940	-0.067964	-0.042145	-0.043931	0.079443
compressionratio	0.086201	-0.984356	0.295541	-0.177888	0.136243	0.127479	-0.019762	0.249786	0.158414	0.181129	...	0.028971	-0.100786	0.005197	0.186110	1.000000	-0.204326	-0.435741	0.324701	0.265201	0.067984
horsepower	-0.091439	0.163926	0.241685	0.126947	-0.153928	0.518686	0.317839	0.353294	0.552623	0.640732	...	0.809769	0.655638	0.573677	0.080940	-0.204326	1.000000	0.131073	-0.801456	-0.770544	0.808139
peakrpm	-0.143058	0.476883	-0.183383	0.247668	-0.109643	-0.039417	0.198461	-0.360469	-0.287242	-0.220012	...	-0.244660	0.014261	-0.254976	-0.067964	-0.435741	0.131073	1.000000	-0.113544	-0.054275	-0.085267
citympg	0.053777	-0.255963	-0.202362	0.012417	0.031697	-0.449581	-0.153487	-0.470414	-0.670909	-0.642704	...	-0.653658	-0.671581	-0.584532	-0.042145	0.324701	-0.801456	-0.113544	1.000000	0.971337	-0.685751
highwaympg	0.055715	-0.191392	-0.254416	0.036330	-0.007170	-0.452220	-0.102026	-0.544082	-0.704662	-0.677218	...	-0.677470	-0.645659	-0.587012	-0.043931	0.265201	-0.770544	-0.054275	0.971337	1.000000	-0.697599
price	-0.173728	-0.105679	0.177926	-0.031835	-0.083976	0.577992	0.324973	0.577816	0.682920	0.759325	...	0.874145	0.526823	0.553173	0.079443	0.067984	0.808139	-0.085267	-0.685751	-0.697599	1.000000
24 rows × 24 columns

numerical_cols=data.select_dtypes(exclude=['object']).columns
data[numerical_cols].head()
CarName	fueltype	aspiration	doornumber	carbody	drivewheel	enginelocation	wheelbase	carlength	carwidth	...	enginesize	fuelsystem	boreratio	stroke	compressionratio	horsepower	peakrpm	citympg	highwaympg	price
0	1	1	0	1	0	2	0	88.6	168.8	64.1	...	130	5	3.47	2.68	9.0	111	5000	21	27	13495.0
1	1	1	0	1	0	2	0	88.6	168.8	64.1	...	130	5	3.47	2.68	9.0	111	5000	21	27	16500.0
2	1	1	0	1	2	2	0	94.5	171.2	65.5	...	152	5	2.68	3.47	9.0	154	5000	19	26	16500.0
3	2	1	0	0	3	1	0	99.8	176.6	66.2	...	109	5	3.19	3.40	10.0	102	5500	24	30	13950.0
4	2	1	0	0	3	0	0	99.4	176.6	66.4	...	136	5	3.19	3.40	8.0	115	5500	18	22	17450.0
5 rows × 24 columns

data.head()
symboling	CarName	fueltype	aspiration	doornumber	carbody	drivewheel	enginelocation	wheelbase	carlength	...	enginesize	fuelsystem	boreratio	stroke	compressionratio	horsepower	peakrpm	citympg	highwaympg	price
0	3	1	1	0	1	0	2	0	88.6	168.8	...	130	5	3.47	2.68	9.0	111	5000	21	27	13495.0
1	3	1	1	0	1	0	2	0	88.6	168.8	...	130	5	3.47	2.68	9.0	111	5000	21	27	16500.0
2	1	1	1	0	1	2	2	0	94.5	171.2	...	152	5	2.68	3.47	9.0	154	5000	19	26	16500.0
3	2	2	1	0	0	3	1	0	99.8	176.6	...	109	5	3.19	3.40	10.0	102	5500	24	30	13950.0
4	2	2	1	0	0	3	0	0	99.4	176.6	...	136	5	3.19	3.40	8.0	115	5500	18	22	17450.0
5 rows × 25 columns

let's split data to train and test data

from sklearn.model_selection import train_test_split
x = data.drop(['price'],axis=1)
y = data['price']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0)
cols=x.columns
cols
Index(['symboling', 'CarName', 'fueltype', 'aspiration', 'doornumber',
       'carbody', 'drivewheel', 'enginelocation', 'wheelbase', 'carlength',
       'carwidth', 'carheight', 'curbweight', 'enginetype', 'cylindernumber',
       'enginesize', 'fuelsystem', 'boreratio', 'stroke', 'compressionratio',
       'horsepower', 'peakrpm', 'citympg', 'highwaympg'],
      dtype='object')
cols=x.columns
X=pd.DataFrame(scale(x))
X.columns=cols
X
symboling	CarName	fueltype	aspiration	doornumber	carbody	drivewheel	enginelocation	wheelbase	carlength	...	cylindernumber	enginesize	fuelsystem	boreratio	stroke	compressionratio	horsepower	peakrpm	citympg	highwaympg
0	1.743470	-1.541331	0.328798	-0.469295	1.130388	-3.050975	1.213330	-0.121867	-1.690772	-0.426521	...	-0.147475	0.074449	0.869568	0.519071	-1.839377	-0.288349	0.174483	-0.262960	-0.646553	-0.546059
1	1.743470	-1.541331	0.328798	-0.469295	1.130388	-3.050975	1.213330	-0.121867	-1.690772	-0.426521	...	-0.147475	0.074449	0.869568	0.519071	-1.839377	-0.288349	0.174483	-0.262960	-0.646553	-0.546059
2	0.133509	-1.541331	0.328798	-0.469295	1.130388	-0.717207	1.213330	-0.121867	-0.708596	-0.231513	...	1.112210	0.604046	0.869568	-2.404880	0.685946	-0.288349	1.264536	-0.262960	-0.953012	-0.691627
3	0.938490	-1.395654	0.328798	-0.469295	-0.884652	0.449677	-0.589081	-0.121867	0.173698	0.207256	...	-0.147475	-0.431076	0.869568	-0.517266	0.462183	-0.035973	-0.053668	0.787855	-0.186865	-0.109354
4	0.938490	-1.395654	0.328798	-0.469295	-0.884652	0.449677	-2.391492	-0.121867	0.107110	0.207256	...	-1.407161	0.218885	0.869568	-0.517266	0.462183	-0.540725	0.275883	0.787855	-1.106241	-1.273900
...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...
200	-1.476452	1.372204	0.328798	-0.469295	-0.884652	0.449677	1.213330	-0.121867	1.721873	1.198549	...	-0.147475	0.339248	0.869568	1.666445	-0.336970	-0.162161	0.250533	0.577692	-0.340094	-0.400490
201	-1.476452	1.372204	0.328798	2.130854	-0.884652	0.449677	1.213330	-0.121867	1.721873	1.198549	...	-0.147475	0.339248	0.869568	1.666445	-0.336970	-0.364062	1.416637	0.367529	-0.953012	-0.837195
202	-1.476452	1.372204	0.328798	-0.469295	-0.884652	0.449677	1.213330	-0.121867	1.721873	1.198549	...	1.112210	1.109571	0.869568	0.926204	-1.232021	-0.338824	0.757535	0.787855	-1.106241	-1.128332
203	-1.476452	1.372204	-3.041381	2.130854	-0.884652	0.449677	1.213330	-0.121867	1.721873	1.198549	...	1.112210	0.435538	-0.126306	-1.183483	0.462183	3.244916	0.047732	-0.683286	0.119594	-0.546059
204	-1.476452	1.372204	0.328798	2.130854	-0.884652	0.449677	1.213330	-0.121867	1.721873	1.198549	...	-0.147475	0.339248	0.869568	1.666445	-0.336970	-0.162161	0.250533	0.577692	-0.953012	-0.837195
205 rows × 24 columns

degrees = [1, 2, 3, 6, 10, 20]
y_train_pred = np.zeros((len(x_train), len(degrees)))
y_test_pred = np.zeros((len(x_test), len(degrees)))
lm = LinearRegression()
lm.fit(x_train, y_train)
LinearRegression()
print(x_train.shape)
print(x_test.shape)
print(y_train_pred.shape)
print(y_test_pred.shape)
(143, 24)
(62, 24)
(143, 6)
(62, 6)
print("R-squared values: \n")

for i, degree in enumerate(degrees):
    train_r2 = round(sklearn.metrics.r2_score(y_train, y_train_pred[:, i]), 2)
    test_r2 = round(sklearn.metrics.r2_score(y_test, y_test_pred[:, i]), 2)
    print("Polynomial degree {0}: train score={1}, test score={2}".format(degree, 
                                                                         train_r2, 
                                                                         test_r2))
R-squared values: 

Polynomial degree 1: train score=-2.83, test score=-2.65
Polynomial degree 2: train score=-2.83, test score=-2.65
Polynomial degree 3: train score=-2.83, test score=-2.65
Polynomial degree 6: train score=-2.83, test score=-2.65
Polynomial degree 10: train score=-2.83, test score=-2.65
Polynomial degree 20: train score=-2.83, test score=-2.65
lm = LinearRegression()
lm.fit(x_train, y_train)

rfe = RFE(lm, n_features_to_select=10)             
rfe = rfe.fit(x_train, y_train)
list(zip(x_train.columns,rfe.support_,rfe.ranking_))
[('symboling', False, 4),
 ('CarName', False, 3),
 ('fueltype', True, 1),
 ('aspiration', False, 5),
 ('doornumber', True, 1),
 ('carbody', True, 1),
 ('drivewheel', True, 1),
 ('enginelocation', True, 1),
 ('wheelbase', False, 2),
 ('carlength', False, 9),
 ('carwidth', True, 1),
 ('carheight', False, 10),
 ('curbweight', False, 14),
 ('enginetype', False, 7),
 ('cylindernumber', False, 12),
 ('enginesize', False, 6),
 ('fuelsystem', False, 11),
 ('boreratio', True, 1),
 ('stroke', True, 1),
 ('compressionratio', True, 1),
 ('horsepower', False, 13),
 ('peakrpm', False, 15),
 ('citympg', True, 1),
 ('highwaympg', False, 8)]
y_pred = rfe.predict(x_test)

r2 = sklearn.metrics.r2_score(y_test, y_pred)
print(r2)
0.6294611235960204
lm = LinearRegression()
lm.fit(x_train, y_train)

rfe = RFE(lm, n_features_to_select=6)             
rfe = rfe.fit(x_train, y_train)

y_pred = rfe.predict(x_test)
r2 = sklearn.metrics.r2_score(y_test, y_pred)
print(r2)
0.6755813909460614
lm = LinearRegression()
scores = cross_val_score(lm, x_train, y_train, scoring='r2', cv=5)
scores  
array([0.65848269, 0.91768233, 0.78648072, 0.88618539, 0.88291521])
KFold
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_val_score(lm, x_train, y_train, scoring='r2', cv=folds)
scores  
array([0.65752016, 0.6696122 , 0.85432835, 0.85278783, 0.81698628])
scores = cross_val_score(lm, x_train, y_train, scoring='neg_mean_squared_error', cv=5)
scores
array([-11026075.53911617,  -7333847.11019093,  -9758707.16845866,
        -8727293.18001828,  -7460306.32815502])
len(x_train.columns)
24
list(range(1, 25))
[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24]
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)

hyper_params = [{'n_features_to_select': list(range(1, 25))}]

lm = LinearRegression()
lm.fit(x_train, y_train)
rfe = RFE(lm)             

model_cv = GridSearchCV(estimator = rfe, 
                        param_grid = hyper_params, 
                        scoring= 'r2', 
                        cv = folds, 
                        verbose = 1,
                        return_train_score=True)      

model_cv.fit(x_train, y_train) 
Fitting 5 folds for each of 24 candidates, totalling 120 fits
GridSearchCV(cv=KFold(n_splits=5, random_state=100, shuffle=True),
             estimator=RFE(estimator=LinearRegression()),
             param_grid=[{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15, 16,
                                                   17, 18, 19, 20, 21, 22, 23,
                                                   24]}],
             return_train_score=True, scoring='r2', verbose=1)
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results.head(5)
mean_fit_time	std_fit_time	mean_score_time	std_score_time	param_n_features_to_select	params	split0_test_score	split1_test_score	split2_test_score	split3_test_score	...	mean_test_score	std_test_score	rank_test_score	split0_train_score	split1_train_score	split2_train_score	split3_train_score	split4_train_score	mean_train_score	std_train_score
0	0.021014	0.000271	0.002369	0.000040	1	{'n_features_to_select': 1}	-0.006625	0.376055	-0.006167	-0.063338	...	0.021175	0.190224	24	0.168576	0.382909	0.208609	0.217423	0.166637	0.228831	0.079722
1	0.020487	0.000620	0.002387	0.000022	2	{'n_features_to_select': 2}	-0.005973	0.314893	0.349076	0.419013	...	0.239105	0.157993	23	0.505495	0.742163	0.505735	0.469580	0.488784	0.542352	0.100786
2	0.019596	0.000188	0.002410	0.000031	3	{'n_features_to_select': 3}	0.036841	0.321798	0.807653	0.676281	...	0.503275	0.283603	18	0.506569	0.753851	0.757283	0.796609	0.777273	0.718317	0.106980
3	0.019129	0.000328	0.002398	0.000046	4	{'n_features_to_select': 4}	0.225504	0.301536	0.825226	0.692782	...	0.546893	0.237721	9	0.818397	0.754621	0.760757	0.799840	0.784524	0.783628	0.023820
4	0.018321	0.000247	0.002429	0.000043	5	{'n_features_to_select': 5}	0.172878	0.285397	0.823715	0.669374	...	0.517165	0.246231	12	0.820992	0.755220	0.761776	0.807229	0.790791	0.787202	0.025397
5 rows × 21 columns

plt.figure(figsize=(16,6))

plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_test_score"])
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_train_score"])
plt.xlabel('number of features')
plt.ylabel('r-squared')
plt.title("Optimal Number of Features")
plt.legend(['test score', 'train score'], loc='upper left')
<matplotlib.legend.Legend at 0x7f1db64b4050>

n_features_optimal = 10

lm = LinearRegression()
lm.fit(x_train, y_train)

rfe = RFE(lm, n_features_to_select=n_features_optimal)             
rfe = rfe.fit(x_train, y_train)

y_pred = lm.predict(x_test)
r2 = sklearn.metrics.r2_score(y_test, y_pred)
print(r2)
0.797957877255957